# **This notebook's best result: val_acc is 0.8779, val_loss is 0.3129**

# **1. Few Preprocessings**
# **2. Model: FastText by Keras**
## **2.1** Change Preprocessings:
- Do lower case 

In [1]:
import numpy as np

import pandas as pd

from collections import defaultdict

import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

np.random.seed(7)

Using TensorFlow backend.


In [2]:
df = pd.read_csv('input/train.csv')
a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
y = np.array([a2c[a] for a in df.author])
y = to_categorical(y)

# 1. **Few Preprocessings**

In traditional NLP tasks, preprocessings play an important role, but...

## **Low-frequency words**
In my experience, fastText is very fast, but I need to delete rare words to avoid overfitting.

**NOTE**:
Some keywords are rare words, such like *Cthulhu* in *Cthulhu Mythos* of *Howard Phillips Lovecraft*.
But these are useful for this task.

## **Removing Stopwords**

Nothing.
To identify author from a sentence, some stopwords play an important role because one has specific usages of them.

## **Stemming and Lowercase**

Nothing.
This reason is the same for stopwords removing.
And I guess some stemming rules provided by libraries is bad for this task because all author is the older author.

## **Cutting long sentence**

Too long documents are cut.

## **Punctuation**

Because I guess each author has unique punctuations's usage in the novel, I separate them from words.

e.g. `Don't worry` -> `Don ' t worry`

## **Is it slow?**

Don't worry! FastText is a very fast algorithm if it runs on CPU. 

# **Let's check character distribution per author**

In [3]:
counter = {name : defaultdict(int) for name in set(df.author)}
for (text, author) in zip(df.text, df.author):
    text = text.replace(' ', '')
    for c in text:
        counter[author][c] += 1

chars = set()
for v in counter.values():
    chars |= v.keys()
    
names = [author for author in counter.keys()]

print('c ', end='')
for n in names:
    print(n, end='   ')
print()
for c in chars:    
    print(c, end=' ')
    for n in names:
        print(counter[n][c], end=' ')
    print()


c HPL   MWS   EAP   
L 249 307 458 
N 345 204 411 
C 439 308 395 
Q 10 7 21 
G 318 246 313 
s 43915 45962 53841 
x 1061 1267 1951 
à 0 0 10 
Π 1 0 0 
W 732 681 739 
F 269 232 383 
î 0 0 1 
e 88259 97515 114885 
: 47 339 176 
u 19519 21025 26311 
f 16272 18351 22354 
t 62235 63142 82426 
I 3480 4917 4846 
; 1143 2662 1354 
c 18338 17911 24127 
k 5204 3707 4277 
M 645 415 1065 
U 94 46 166 
Y 111 234 282 
v 6529 7948 9624 
ô 0 0 8 
α 2 0 0 
O 503 282 414 
R 237 385 258 
ñ 7 0 0 
i 44250 46080 60952 
è 0 0 15 
d 33366 35315 36862 
Ν 1 0 0 
æ 10 0 36 
ä 6 0 1 
Υ 1 0 0 
B 533 395 835 
Å 1 0 0 
δ 2 0 0 
H 741 669 864 
m 17622 20471 22792 
y 12534 14877 17001 
V 67 57 156 
ë 12 0 0 
K 176 35 86 
Æ 4 0 1 
Z 51 2 23 
h 42770 43738 51580 
n 50879 50291 62636 
J 210 66 164 
w 15554 16062 17507 
ç 0 0 1 
q 779 677 1030 
" 513 1469 2987 
r 40590 44042 51221 
ê 2 0 28 
? 169 419 510 
l 30273 27819 35371 
' 1710 476 1334 
â 0 0 6 
b 10636 9611 13245 
o 50996 53386 67145 
A 1167 943 1258 
ï 7 0 0 
g 1

# **Summary of character distribution**

- HPL and EAP used non ascii characters like a `ä`.
- The number of punctuations seems to be good feature


# **Preprocessing**

My preproceeings are 

- Separate punctuation from words
- Remove lower frequency words ( <= 2)
- Cut a longer document which contains `256` words

In [4]:
def preprocess(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

In [5]:
def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

In [6]:
min_count = 2

docs = create_docs(df)
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)

# **2. Model: FastText by Keras**

FastText is very fast and strong baseline algorithm for text classification based on Continuous Bag-of-Words model a.k.a Word2vec.

FastText contains only three layers:

1. Embeddings layer: Input words (and word n-grams) are all words in a sentence/document
2. Mean/AveragePooling Layer: Taking average vector of Embedding vectors
3. Softmax layer

There are some implementations of FastText:

- Original library provided by Facebook AI research: https://github.com/facebookresearch/fastText
- Keras: https://github.com/fchollet/keras/blob/master/examples/imdb_fasttext.py
- Gensim: https://radimrehurek.com/gensim/models/wrappers/fasttext.html

Original Paper: https://arxiv.org/abs/1607.01759 : More detail information about fastText classification model

# My FastText parameters are:

- The dimension of word vector is 20
- Optimizer is `Adam`
- Inputs are words and word bi-grams
  - you can change this parameter by passing the max n-gram size to argument of `create_docs` function.


In [7]:
input_dim = np.max(docs) + 1
embedding_dims = 20

In [8]:
def create_model(embedding_dims=20, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [9]:
epochs = 25
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/25
15663/15663 [==============================] - 40s 3ms/step - loss: 1.0675 - acc: 0.4077 - val_loss: 1.0298 - val_acc: 0.4545
Epoch 2/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.9330 - acc: 0.6121 - val_loss: 0.8592 - val_acc: 0.7163
Epoch 3/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.7250 - acc: 0.7874 - val_loss: 0.7072 - val_acc: 0.7574
Epoch 4/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.5657 - acc: 0.8475 - val_loss: 0.6096 - val_acc: 0.7870
Epoch 5/25
15663/15663 [==============================] - 38s 2ms/step - loss: 0.4523 - acc: 0.8801 - val_loss: 0.5402 - val_acc: 0.8041
Epoch 6/25
15663/15663 [==============================] - 37s 2ms/step - loss: 0.3663 - acc: 0.9068 - val_loss: 0.4877 - val_acc: 0.8195
Epoch 7/25
15663/15663 [==============================] - 37s 2ms/step - loss: 0.2987 - acc: 0.9283 - val_loss: 0.4506 - val_acc:

## Result

- Best `val_loss` is 0.3409
- Best `val_acc` is 0.8700

## **2.1** Change Preprocessings

Next, I change some parameters and preprocessings to improve fastText model.

###  **2.1.1 Do lower case**

In [10]:
docs = create_docs(df)
tokenizer = Tokenizer(lower=True, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=True, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)

input_dim = np.max(docs) + 1

In [11]:
epochs = 25
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/25
15663/15663 [==============================] - 40s 3ms/step - loss: 1.0664 - acc: 0.4097 - val_loss: 1.0258 - val_acc: 0.4221
Epoch 2/25
15663/15663 [==============================] - 39s 3ms/step - loss: 0.9242 - acc: 0.6185 - val_loss: 0.8485 - val_acc: 0.6813
Epoch 3/25
15663/15663 [==============================] - 42s 3ms/step - loss: 0.7203 - acc: 0.7862 - val_loss: 0.6999 - val_acc: 0.7638
Epoch 4/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.5662 - acc: 0.8396 - val_loss: 0.5998 - val_acc: 0.7962
Epoch 5/25
15663/15663 [==============================] - 39s 2ms/step - loss: 0.4558 - acc: 0.8771 - val_loss: 0.5334 - val_acc: 0.8105
Epoch 6/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.3723 - acc: 0.9021 - val_loss: 0.4812 - val_acc: 0.8325
Epoch 7/25
15663/15663 [==============================] - 38s 2ms/step - loss: 0.3059 - acc: 0.9240 - val_loss: 0.4437 - val_acc:

## Result

- Best `val_loss` is 0.3129
- Best `val_acc` is 0.8787

### **2.1.2 Use less words**
I decrease the length of sentence to train model.

In [12]:
docs = create_docs(df)
tokenizer = Tokenizer(lower=True, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=True, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = 128

docs = pad_sequences(sequences=docs, maxlen=maxlen)

input_dim = np.max(docs) + 1

In [13]:
epochs = 25
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/25
15663/15663 [==============================] - 38s 2ms/step - loss: 1.0360 - acc: 0.4549 - val_loss: 0.9477 - val_acc: 0.5723
Epoch 2/25
15663/15663 [==============================] - 39s 3ms/step - loss: 0.7791 - acc: 0.7407 - val_loss: 0.7118 - val_acc: 0.7406
Epoch 3/25
15663/15663 [==============================] - 39s 3ms/step - loss: 0.5466 - acc: 0.8446 - val_loss: 0.5827 - val_acc: 0.7893
Epoch 4/25
15663/15663 [==============================] - 41s 3ms/step - loss: 0.4029 - acc: 0.8922 - val_loss: 0.5080 - val_acc: 0.8098
Epoch 5/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.3053 - acc: 0.9240 - val_loss: 0.4545 - val_acc: 0.8271
Epoch 6/25
15663/15663 [==============================] - 40s 3ms/step - loss: 0.2333 - acc: 0.9461 - val_loss: 0.4189 - val_acc: 0.8368
Epoch 7/25
15663/15663 [==============================] - 37s 2ms/step - loss: 0.1794 - acc: 0.9631 - val_loss: 0.3957 - val_acc:

In [16]:
test_df = pd.read_csv('input/test.csv')
docs = create_docs(test_df)
docs = tokenizer.texts_to_sequences(docs)
docs = pad_sequences(sequences=docs, maxlen=maxlen)
y = model.predict_proba(docs)

result = pd.read_csv('input/sample_submission.csv')
for a, i in a2c.items():
    result[a] = y[:, i]

In [15]:
# to_submit=result